In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "input"       )
dir_data     = os.path.join(dir_inputs , "REMMdata"    )

AnalysisAreasInput = os.path.join(dir_inputs, r"Parcel-Redevelopment.gdb\CentersTOD_ResidentialCapacity_Areas")
ClassParameters    = os.path.join(dir_inputs, r"class_parameters.csv"                       )

ParcelsGDB = "parcels.gdb"
Parcels    = os.path.join(dir_process, ParcelsGDB + "\Parcels"              )

ProcessGDB = "process.gdb"

#name of new data features
AnalysisAreas         = os.path.join(dir_process, ProcessGDB + "\AnalysisAreas"        )
AnalysisAreas_ByName  = os.path.join(dir_process, ProcessGDB + "\AnalysisAreas_ByName" )
AnalysisAreas_ByClass = os.path.join(dir_process, ProcessGDB + "\AnalysisAreas_ByClass")
AnalysisAreas_IDs     = os.path.join(dir_process, ProcessGDB + "\AnalysisAreas_IDs"    )
ParcelsAA_join        = os.path.join(dir_process, ProcessGDB + "\ParcelsAA_join"       )
ParcelsAA             = os.path.join(dir_process, ProcessGDB + "\ParcelsAA"            )

#name of new data features
AnalysisAreas_shp         = os.path.join(dir_process, "AnalysisAreas.shp"        )
AnalysisAreas_ByName_shp  = os.path.join(dir_process, "AnalysisAreas_ByName.shp" )
AnalysisAreas_ByClass_shp = os.path.join(dir_process, "AnalysisAreas_ByClass.shp")
AnalysisAreas_IDs_shp     = os.path.join(dir_process, "AnalysisAreas_IDs.shp"    )
Parcels_shp               = os.path.join(dir_process, "Parcels.shp"              )
ParcelsAA_shp             = os.path.join(dir_process, "ParcelsAA.shp"            )

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)

print(AnalysisAreasInput)

E:\GitHub\Parcel-Redevelopment-Potential
E:\GitHub\Parcel-Redevelopment-Potential\input\Parcel-Redevelopment.gdb\CentersTOD_ResidentialCapacity_Areas


# Prep Analysis Areas

In [4]:
#Create Analysis Area Layer in Intermediate Files and Fill with a AAName that combines Name and TOD status

sAreaBlankName  = 'No Area Name'

sAAName  = "AnalysisAreaName" #field for calculated Analysis Area Name
sAAClass = "AnalysisAreaClass" #field for calculated Analysis Area Class

#if processing geodatabase doesn't exist, create it
print("Checking if " + ProcessGDB + " exists...")
if not arcpy.Exists(os.path.join(dir_process, ProcessGDB)):
    print("Creating " + ProcessGDB + "...")
    arcpy.management.CreateFileGDB(dir_process, ProcessGDB)
else:
    print(ProcessGDB + " exists...")

print ("Creating Analysis Input Area intermediate layer in " + ProcessGDB + "...")
deleteIfExists(AnalysisAreas)
arcpy.management.Copy(AnalysisAreasInput, AnalysisAreas)

## Subroutines

print ("Adding " + sAAName + " field to " + AnalysisAreas + "...")
arcpy.AddField_management(AnalysisAreas, sAAClass, "TEXT", 50)

areaname_codeblock = """def createAreaName(centername):
    return centername"""

#create calculated Class for analysis area combining center Class and tod status
centerClass_codeblock = """def createClassName(centerClass):
   return centerClass"""


print ("Calculating " + sAAName + " field...")
arcpy.CalculateField_management(AnalysisAreas,
                                sAAName,
                                "createAreaName(!ClassCode!)",
                                "PYTHON_9.3",
                                areaname_codeblock)
print ("Calculating " + sAAClass + " field...")
arcpy.CalculateField_management(AnalysisAreas,
                                sAAClass,
                                "createClassName(!ClassCode!)",
                                "PYTHON_9.3",
                                centerClass_codeblock)

print ("Creating " + AnalysisAreas_ByName + " intermediate layer in " + ProcessGDB + "...")
deleteIfExists(AnalysisAreas_ByName)
arcpy.management.Dissolve(AnalysisAreas, AnalysisAreas_ByName,  sAAName)

print ("Creating " + AnalysisAreas_ByClass + " intermediate layer in " + ProcessGDB + "...")
deleteIfExists(AnalysisAreas_ByClass)
arcpy.management.Dissolve(AnalysisAreas, AnalysisAreas_ByClass, sAAClass)

print ("All Done")

Checking if process.gdb exists...
process.gdb exists...
Creating Analysis Input Area intermediate layer in process.gdb...
Adding AnalysisAreaName field to E:\GitHub\Parcel-Redevelopment-Potential\intermediate\process.gdb\AnalysisAreas...
Calculating AnalysisAreaName field...
Calculating AnalysisAreaClass field...
Creating E:\GitHub\Parcel-Redevelopment-Potential\intermediate\process.gdb\AnalysisAreas_ByName intermediate layer in process.gdb...
Creating E:\GitHub\Parcel-Redevelopment-Potential\intermediate\process.gdb\AnalysisAreas_ByClass intermediate layer in process.gdb...
All Done


In [5]:

#Areas data
df_AnalysisAreas = pd.DataFrame()

cursorName = arcpy.SearchCursor(AnalysisAreas_ByName)
row = cursorName.next()
while row:
    #print(row.getValue(sAAName))
    df_AnalysisAreas = df_AnalysisAreas.append({sAAName : row.getValue(sAAName)},ignore_index=True)
    row = cursorName.next()  
    
df_AnalysisAreas.index.name = 'AreaID'
df_AnalysisAreas = df_AnalysisAreas.reset_index()
df_AnalysisAreas.to_csv(os.path.join(dir_results, r'areas.csv'),index=False)
display(df_AnalysisAreas)

,AreaID,AnalysisAreaName
0,0,a1
1,1,a2
2,2,a3
3,3,a4
4,4,a5
5,5,b1
6,6,b2
7,7,b3
8,8,b4
9,9,b5


In [6]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder
0,f4,Multifamily Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,29
1,f3,Multifamily City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,28
2,f2,Multifamily Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27
3,f1,Multifamily Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,26
4,g4,Mixed-Use Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,33
5,g3,Mixed-Use City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32
6,g2,Mixed-Use Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31
7,g1,Mixed-Use Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,30
8,h4,Mixed Residential Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,37
9,h3,Mixed Residential City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,36


In [9]:
sdf_AA

,OBJECTID,ClassName,code,ClassCode,AnalysisAreaClass,AnalysisAreaName,SHAPE
0,1,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{""curveRings"": [[[444099.3367999997, 4452551.3..."
1,2,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{""curveRings"": [[[438376.81769999955, 4459637...."
2,3,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{""curveRings"": [[[430849.71740000043, 4469533...."
3,4,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{""curveRings"": [[[423912.7335000001, 4474824.7..."
4,5,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{""curveRings"": [[[423670.5016999999, 4485868.4..."
...,...,...,...,...,...,...,...
13204,13205,,h1,h1,h1,h1,"{""rings"": [[[429644.14269999973, 4481668.1929]..."
13205,13206,LRT|NA,d4,LRT|NA,LRT|NA,LRT|NA,"{""rings"": [[[415404.0895999996, 4492683.829600..."
13206,13207,LRT|NA,h1,LRT|NA,LRT|NA,LRT|NA,"{""rings"": [[[415404.0895999996, 4492683.829600..."
13207,13208,NONTOD|City Center,b3,NONTOD|City Center,NONTOD|City Center,NONTOD|City Center,"{""rings"": [[[410819.7604, 4557518.294199999], ..."


In [11]:
#create layer with just IDs to make union cleaner

df_ClassIDs = df_ClassParam[['ClassID','ClassDescription']]
#display(df_ClassIDs)

sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
#display(sdf_AnalysisAreas)

sdf_AAwClassID =  pd.DataFrame.merge(sdf_AA,df_ClassIDs,left_on='ClassCode', right_on='ClassID')
#display(sdf_AAwID)

sdf_AAwClassIDwNameID =  pd.DataFrame.merge(sdf_AAwClassID,df_AnalysisAreas,on=sAAName)
#display(sdf_AAwClassIDwNameID)

sdf_AAIDs = sdf_AAwClassIDwNameID[['AreaID','ClassID','SHAPE']]
#display(sdf_AAIDs)

#arcpy.Delete_management(AnalysisAreas_IDs)
deleteIfExists(AnalysisAreas_IDs)
sdf_AAIDs.spatial.to_featureclass(location=AnalysisAreas_IDs)
print('All Done')

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\geo\_accessor.py:1211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[col] = GeoArray(self._data[col])
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


All Done


In [12]:
sdf_AAwClassIDwNameID

,OBJECTID,ClassName,code,ClassCode,AnalysisAreaClass,AnalysisAreaName,SHAPE,ClassID,ClassDescription,AreaID
0,1,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{'curveRings': [[[444099.3367999997, 4452551.3...",CRT|NA,CRT|NA,17
1,2,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{'curveRings': [[[438376.81769999955, 4459637....",CRT|NA,CRT|NA,17
2,3,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{'curveRings': [[[430849.71740000043, 4469533....",CRT|NA,CRT|NA,17
3,4,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{'curveRings': [[[423912.7335000001, 4474824.7...",CRT|NA,CRT|NA,17
4,5,CRT|NA,,CRT|NA,CRT|NA,CRT|NA,"{'curveRings': [[[423670.5016999999, 4485868.4...",CRT|NA,CRT|NA,17
...,...,...,...,...,...,...,...,...,...,...
13204,9612,,h3,h3,h3,h3,"{'rings': [[[422994.2566999998, 4515221.621999...",h3,Mixed Residential City/TOD,39
13205,9613,,h3,h3,h3,h3,"{'rings': [[[424185.25090000033, 4515427.8761]...",h3,Mixed Residential City/TOD,39
13206,9614,,h3,h3,h3,h3,"{'rings': [[[422667.19579999987, 4515263.19679...",h3,Mixed Residential City/TOD,39
13207,9615,,h3,h3,h3,h3,"{'rings': [[[422555.8554999996, 4516046.902699...",h3,Mixed Residential City/TOD,39


# Intersect Analysis Areas with Parcels

In [13]:
deleteIfExists(ParcelsAA_join)

#intersection parcels with analysis area types
arcpy.analysis.Intersect([AnalysisAreas_IDs,Parcels], ParcelsAA_join)

<Result 'E:\\GitHub\\Parcel-Redevelopment-Potential\\intermediate\\process.gdb\\ParcelsAA_join'>

In [14]:


print ("Calculating area...")
arcpy.AddField_management(ParcelsAA_join, "piece_acres" , "DOUBLE")
arcpy.CalculateField_management(ParcelsAA_join,
                                "PieceAcres",
                                "!shape.area@acres!",
                                "PYTHON_9.3")
arcpy.AddField_management(ParcelsAA_join, "piece_portion" , "DOUBLE")
arcpy.CalculateField_management(ParcelsAA_join,
                                "PiecePortion",
                                "$feature.PieceAcres / $feature.ParcelAcres", "ARCADE")

print ("Done")

Calculating area...
Done


In [15]:
sdf_ParcelsAA = pd.DataFrame.spatial.from_featureclass(ParcelsAA_join)
sdf_ParcelsAA.columns

Index(['OBJECTID', 'FID_AnalysisAreas_IDs', 'area_id', 'class_id',
       'FID_Parcels', 'parcel_id', 'parcel_id_remm', 'basebldg', 'parcel_sqft',
       'parcel_acre', 'county_id', 'max_dua', 'max_far', 'max_height',
       'lu_type', 'bldgtype', 'yearbuilt', 'resunits', 'job_spaces', 'sf_res',
       'sf_com', 'sfvalue_res', 'sfvalue_com', 'value_res', 'value_com',
       'ParcelAcres', 'PieceAcres', 'PiecePortion', 'SHAPE'],
      dtype='object')

In [19]:
#adjust parcel values by portion of parcel in parcel piece
sdf_ParcelsAA['parcel_sqft'] = sdf_ParcelsAA['parcel_sqft'] * sdf_ParcelsAA['piece_portion']
sdf_ParcelsAA['resunits'   ] = sdf_ParcelsAA['resunits'   ] * sdf_ParcelsAA['piece_portion']
sdf_ParcelsAA['job_spaces' ] = sdf_ParcelsAA['job_spaces' ] * sdf_ParcelsAA['piece_portion']
sdf_ParcelsAA['sf_res'     ] = sdf_ParcelsAA['sf_res'     ] * sdf_ParcelsAA['piece_portion']
sdf_ParcelsAA['sf_com'     ] = sdf_ParcelsAA['sf_com'     ] * sdf_ParcelsAA['piece_portion']
sdf_ParcelsAA['value_res'  ] = sdf_ParcelsAA['value_res'  ] * sdf_ParcelsAA['piece_portion']
sdf_ParcelsAA['value_com'  ] = sdf_ParcelsAA['value_com'  ] * sdf_ParcelsAA['piece_portion']

#make sure acres used in future is just for piece of parcel, not all
sdf_ParcelsAA['acres'] = sdf_ParcelsAA['piece_acres']
display(sdf_ParcelsAA)

display("exporting Parcels")

sdf_ParcelsAA.spatial.to_featureclass(location=ParcelsAA)

,objectid,fid_analysis_areas_i_ds,area_id,class_id,fid_parcels,parcel_id,parcel_id_remm,basebldg,parcel_sqft,parcel_acre,...,sf_com,sfvalue_res,sfvalue_com,value_res,value_com,parcel_acres,piece_acres,piece_portion,SHAPE,acres
0,1,1,17,CRT|NA,19736,645747.0,645747.0,1.0,301.454973,3.220936,...,163.207152,0.0,0.0,0.0,0.0,3.220934,0.415623,0.129038,"{""curveRings"": [[[444099.3367999997, 4452551.3...",0.415623
1,2,1,17,CRT|NA,24447,598653.0,598653.0,1.0,0.001070,5.987768,...,0.000495,0.0,0.0,0.0,0.0,5.987770,0.009585,0.001601,"{""curveRings"": [[[444148.9570000004, 4452591.1...",0.009585
2,3,1,17,CRT|NA,37976,598283.0,598283.0,0.0,657.394311,0.557976,...,NaN,NaN,NaN,NaN,NaN,0.557976,0.167491,0.300175,"{""rings"": [[[443767.41139999963, 4452576.75919...",0.167491
3,4,1,17,CRT|NA,131771,642522.0,642522.0,0.0,6.328201,0.124929,...,NaN,NaN,NaN,NaN,NaN,0.124928,0.013137,0.105158,"{""curveRings"": [[[444148.93620000035, 4452591....",0.013137
4,5,1,17,CRT|NA,155153,646774.0,646774.0,1.0,8.135949,2.883254,...,4.838658,0.0,0.0,0.0,0.0,2.883256,0.115797,0.040162,"{""curveRings"": [[[443768.3559999997, 4452501.2...",0.115797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027903,1027904,11804,37,h1,648134,549671.0,549671.0,0.0,231.626841,0.171895,...,NaN,NaN,NaN,NaN,NaN,0.171895,0.053960,0.313916,"{""rings"": [[[416285.11959999986, 4572384.52040...",0.053960
1027904,1027905,11804,37,h1,665078,546641.0,546641.0,0.0,231.626841,0.171895,...,NaN,NaN,NaN,NaN,NaN,0.171895,0.053960,0.313916,"{""rings"": [[[416285.11959999986, 4572384.52040...",0.053960
1027905,1027906,11804,37,h1,665079,545690.0,545690.0,0.0,231.626841,0.171895,...,NaN,NaN,NaN,NaN,NaN,0.171895,0.053960,0.313916,"{""rings"": [[[416285.11959999986, 4572384.52040...",0.053960
1027906,1027907,11804,37,h1,685145,548151.0,548151.0,0.0,231.626841,0.171895,...,NaN,NaN,NaN,NaN,NaN,0.171895,0.053960,0.313916,"{""rings"": [[[416285.11959999986, 4572384.52040...",0.053960


'exporting Parcels'

'E:\\GitHub\\Parcel-Redevelopment-Potential\\intermediate\\process.gdb\\ParcelsAA'

In [20]:
sdf_ParcelsAA.groupby(['area_id','class_id'], as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "acres":[np.sum]})

,area_id,class_id,parcel_id,job_spaces,resunits,acres
,,,size,sum,sum,sum
0,0,a1,13095.0,226.688223,1392.344480,18537.668449
1,1,a2,12941.0,451.564918,3169.517601,15433.671502
2,2,a3,16402.0,1396.846492,5504.083008,19063.441789
3,3,a4,11839.0,977.832660,3610.321731,8915.357612
4,4,a5,5311.0,974.335819,2813.550268,5889.372093
5,5,b1,8835.0,1077.906305,2917.467340,10481.991379
6,6,b2,21306.0,828.742176,12267.218797,14656.221448
7,7,b3,23158.0,2457.831194,11072.366567,8799.637586
8,8,b4,18373.0,1627.819803,10161.046634,9472.022185


In [21]:
#export to shp files for better drawing in Jupyter

deleteIfExists(AnalysisAreas_shp)
print ("Exporting " + AnalysisAreas_shp + "...")
arcpy.conversion.FeatureClassToShapefile(AnalysisAreas, dir_process)

deleteIfExists(AnalysisAreas_IDs_shp)
print ("Exporting " + AnalysisAreas_IDs_shp + "...")
arcpy.conversion.FeatureClassToShapefile(AnalysisAreas_IDs, dir_process)

deleteIfExists(ParcelsAA_shp)
print ("Exporting " + ParcelsAA_shp + "...")
arcpy.conversion.FeatureClassToShapefile(ParcelsAA, dir_process) 

display("done!")

Exporting E:\GitHub\Parcel-Redevelopment-Potential\intermediate\AnalysisAreas.shp...
Exporting E:\GitHub\Parcel-Redevelopment-Potential\intermediate\AnalysisAreas_IDs.shp...
Exporting E:\GitHub\Parcel-Redevelopment-Potential\intermediate\ParcelsAA.shp...


'done!'